In [2]:
import astropy.units as u
from radio_beam import Beam, commonbeam, Beams
from astropy.io import fits
import warnings
from astropy.wcs import WCS
import astropy.units as u
from astropy.convolution import convolve
import matplotlib.pyplot as plt
import numpy as np
from astropy.visualization import simple_norm
from astropy.stats import mad_std
from astropy import visualization

In [6]:
path_to_mosaic = '/Users/nbudaiev/UFL Dropbox/Nazar Budaiev/Research/ACES/12m_continuum_commonbeam_circular_reimaged_mosaic.fits'
mosaic_hdul = fits.open(path_to_mosaic)
wcs_aces = WCS(mosaic_hdul[0].header)
aces_beam = Beam.from_fits_header(mosaic_hdul[0].header)
path_to_meerkat = '/Users/nbudaiev/UFL Dropbox/Nazar Budaiev/Research/ACES/MeerKAT_Galactic_Centre_1284MHz-StokesI.fits'
meerkat_hdul = fits.open(path_to_meerkat)
meerkat_beam = Beam.from_fits_header(meerkat_hdul[0].header)
#common_beam = commonbeam(aces_beam, meerkat_beam)
beams =  Beams(beams=[meerkat_beam,aces_beam])
common_beam = beams.common_beam()

hdr_aces = mosaic_hdul[0].header

In [7]:
aces_scale = wcs_aces.proj_plane_pixel_scales()[0]

kernel = common_beam.deconvolve(aces_beam).as_kernel(aces_scale)

conv_aces = convolve(mosaic_hdul[0].data, kernel, preserve_nan=True)

area_rat = (common_beam.sr/aces_beam.sr).value

conv_aces_adj = conv_aces * area_rat

In [14]:
common_beam == meerkat_beam

True

In [11]:
np.nanmax(conv_aces)

np.float64(2.3190879600347696)

In [12]:
np.nanmax(conv_aces_adj)

np.float64(5.66166539332602)

In [13]:
np.nanmax(mosaic_hdul[0].data)

np.float64(4.448284615037065)

In [8]:
hdr_aces.update(common_beam.to_header_keywords())

In [9]:
fits.writeto('12m_continuum_commonbeam_circular_reimaged_mosaic_convolved_with_meerkat.fits', conv_aces_adj, hdr_aces, overwrite = True)